In [2]:
import numpy as np
import pandas as pd

# 1. Create a Pandas DataFrame Object

Pandas ``DataFrame`` can be constructed in a variety of ways using `pandas.DataFrame` function. Check the [Documentation](https://pandas.pydata.org/docs/reference/frame.html) for more details.

```python
pd.DataFrame(data, index=index, columns=columns)
```

Where ``index`` and ``columns`` are optional arguments.


## 1.1 Construct a DataFrame from a 2D Array

First, let's create a 2D array with some randomly numbers. 


- The `numpy.random.randint` function (check [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) for details)  generates an array of numbers from an range.

- The `numpy.random.seed` function (check [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html) for details) sets **a global seed** to ensure reproductivity. The global seed affects all `numpy.random` functions in this notebook until it is changed.



In [3]:
np.random.seed(2025)             # a random seed ensure reproductivity

arr = np.random.randint(low = 0, high = 10, size = (3, 4))   # generate a 3*4 array with integers randomly selected from [0,10)

arr

array([[2, 8, 3, 3],
       [0, 6, 8, 5],
       [1, 8, 5, 7]])

To control a specific random function only, use `numpy.random.RandState` class (check [documentation](https://numpy.org/doc/stable/reference/random/legacy.html#numpy.random.RandomState) for details) and its associated function. The above codes can be rewritten in this way:

```python
rng = np.random.RandomState(2025)     
arr = rng.randint(low = 0, high = 10, size = (3, 4)) 
arr
```

Next, let's convert the 2D array as a data frame.

In [4]:
arr_df = pd.DataFrame(data = arr, 
                      columns=['A', 'B', 'C', 'D'],
                      index = ['r1','r2','r3'])  

arr_df    # element-wise computation applies to data frames as well

,A,B,C,D
r1,2,8,3,3
r2,0,6,8,5
r3,1,8,5,7


## 1.2 Construct a DataFrame from a Dictionary

Note that each key-value pair should be of the same length.

- This approach can also be applied to a list of dictionaries. Alternatively, use `pandas.read_json` function to read a JSON file as Data Frame (check [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html)) directly.

In [5]:
# create a dictionary

state_dict = {'state':['California','Texas','New York','Florida', 'Illinois'],
              'pop':[38332521,26448193,19651127,19552860,12882135],
              'area':[423967,695662,141297,170312,149995]}
state_dict

{'state': ['California', 'Texas', 'New York', 'Florida', 'Illinois'],
 'pop': [38332521, 26448193, 19651127, 19552860, 12882135],
 'area': [423967, 695662, 141297, 170312, 149995]}

In [6]:
# create a dataframe based on the dictionary

state_df = pd.DataFrame(data = state_dict)  # keys become col names

state_df  

,state,pop,area
0,California,38332521,423967
1,Texas,26448193,695662
2,New York,19651127,141297
3,Florida,19552860,170312
4,Illinois,12882135,149995


In [7]:
state_df.shape    # check number of rows and columns

(5, 3)

In [8]:
state_df.columns  # check column labels

Index(['state', 'pop', 'area'], dtype='object')

In [9]:
state_df.index    # check row labels, you can convert it as list  

RangeIndex(start=0, stop=5, step=1)

In [10]:
state_df.set_index(keys='state',inplace= True)    # set column 'state' as index, use state_df.reset_index() to undo it

state_df

,pop,area
state,,
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


## 1.3 Save and Read a DataFrame  

I'd like to save the file in my work folder `Python_Codes`, where this notebook is located.  

- We shall use the `pandas.DataFrame.to_csv` function (check the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) to learn more).

- You can either specify the relative path (starting from current work folder, i.e., `df.csv`) or the full path (starting from root folder, e.g., `/Users/jingliu/OneDrive - Hong Kong Baptist University/ECON3105/2025Fall/Python_Codes/df.csv`).  Relative path is recommended.

- Note the path in your computer may be different, depending on the system and file directories. 

In [11]:
state_df.to_csv('df.csv', index = False)      # relative path is recommended

<font color=red>***Exercise 1: Your Codes Here***</font>  

Can you read the csv file back with the `pandas.read_csv` function (check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html))? Please save it in a variable `state_df2` and display the values.    

- You may specify either relative or full path.

In [26]:
state_df2 = pd.read_csv("df.csv")


<font color=green>***Notes for Google Colab***</font>

If you are using **Google Colab**, please follow the following steps to save/read data into/from **Google Drive**:

1.  Allow Colab to access your Google Drive first (go to `Files` > `Mount Drive`, below code block will pop up, just run it).

```python
from google.colab import drive
drive.mount('/content/drive')
```

2.  Specify the full path ti the file in your Google Drive when writing/reading.   For example, we use below codes to write/read the file into/from the folder `ECON3105` in my Google Drive. You may modify the path according to the file location in your Google Drive.
```python
state_df.to_csv('/content/drive/MyDrive/ECON3105/df.csv')    # to save the file to Google Drive
pd.read_csv('/content/drive/MyDrive/ECON3105/df.csv')        # to read it back from Google Drive
```` 

# 2. Operations on DataFrames

## 2.1 Select and Sreate Columns

In [12]:
state_df['area']            # select a column by column name

state
California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [13]:
state_df.area               # alternative approach,  NOT recommended due to the .method (try state_df.pop)

state
California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [14]:
state_df[['area', 'pop']]   # use double square brackets to select multiple columns

,area,pop
state,,
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Which state is the most crowded? We shall compute the density for each state first.   

In [15]:
state_df['density'] = state_df['pop'] / state_df['area']   # create a new col 'density' based on existing cols

state_df

,pop,area,density
state,,,
California,38332521,423967,90.413926
Texas,26448193,695662,38.018740
New York,19651127,141297,139.076746
Florida,19552860,170312,114.806121
Illinois,12882135,149995,85.883763


## 2.2 Indexer: loc and iloc 

* ``loc`` references the explicit row/column names
* ``iloc`` references the implicit index - position of the row/col

In [27]:
state_df

,pop,area,density
state,,,
California,38332521,423967,120.000000
Texas,26448193,695662,38.018740
New York,19651127,141297,139.076746
Florida,19552860,170312,114.806121
Illinois,12882135,149995,85.883763


In [16]:
state_df.loc['New York','pop']       # select pop value for New York

19651127

In [17]:
state_df.iloc[2,0]                   # same as above (note python index starts from 0)            

19651127

In [18]:
state_df.loc['California', 'density'] = 120      # replace a value, same as state_df.iloc[0, 2] = 120     

state_df

,pop,area,density
state,,,
California,38332521,423967,120.000000
Texas,26448193,695662,38.018740
New York,19651127,141297,139.076746
Florida,19552860,170312,114.806121
Illinois,12882135,149995,85.883763


In [36]:
state_df.loc[state_df['density'] > 100, ['pop', 'area']]     # conditional selection  

,pop,area
state,,
California,38332521,423967
New York,19651127,141297
Florida,19552860,170312


<font color=red>***Exercise 2: Your Codes Here***</font>  

* Task 1: please create a new column ``pop_in_thousands`` to represent the population size in thousands. You may use ``//`` operator to return integer result (e.g., `3200//1000` will return `3`).
* Task 2: create another new column ``state_type``, its value is `'big'` for states with `density` above 100,  and `'small'` for states with `density` no more than 100.  *Hints:  for value assignment, you may use conditional selection, or list comprehension, or use `np.where` (check the [documentation](https://numpy.org/doc/stable/reference/generated/numpy.where.html) for details).*

In [ ]:
# deleted after use 
state_df

,pop,area,density
state,,,
California,38332521,423967,120.000000
Texas,26448193,695662,38.018740
New York,19651127,141297,139.076746
Florida,19552860,170312,114.806121
Illinois,12882135,149995,85.883763


In [45]:
# state_df = pd.DataFrame

state_df["pop_in_thousands"] = state_df["pop"]//1000

state_df

state_df["state_type"] = np.where(state_df["density"]>100,"big","small")


In [46]:
state_df

,pop,area,density,pop_in_thousands,state_type
state,,,,,
California,38332521,423967,120.000000,38332,big
Texas,26448193,695662,38.018740,26448,small
New York,19651127,141297,139.076746,19651,big
Florida,19552860,170312,114.806121,19552,big
Illinois,12882135,149995,85.883763,12882,small


## 2.3 Handle Missing Data

``Pandas`` treats ``None`` and ``NaN`` as interchangeable to indicate missing values. ``Pandas`` also converts between them where appropriate.

In [20]:
df = pd.DataFrame([[1, np.nan,  2, None],
                   [2,      3,  5, None],
                   [None,   4,  6, None]])
df

,0,1,2,3
0,1.0,NaN,2,None
1,2.0,3.0,5,None
2,NaN,4.0,6,None


**Detect missing values**

The `pandas.dataframe.isnull()` function (check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html) for details) returns a boolean same-sized object indicating if the values are NA. 

In [21]:
df.isnull()         # check by cell, opposite to df.notnull()

,0,1,2,3
0,False,True,False,True
1,False,False,False,True
2,True,False,False,True


To check if a column contain any missing value.

- The `pandas.dataframe.any()` function (check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.any.html) for details) check whether at least one element over an axis (default 0, across rows) is `True` or equivalent (e.g. non-zero or non-empty).

In [22]:
df.isnull().any()   # use .any(axis = 1) to check across cols (any rows contains NA)

0     True
1     True
2    False
3     True
dtype: bool

**Drop missing values**

The `pandas.DataFrame.dropna()` drops a row (default) or a column if that row/column contains missing values.  Check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) for details.

- We can only drop full rows or full columns (not a single value) from a `DataFrame`.

In [23]:
df.dropna()           # drop rows with any NA, same as df.dropna(axis=0)

,0,1,2,3


In [24]:
df.dropna(axis = 1)   # drop columns with any NA 

,2
0,2
1,5
2,6


**Fill missing values**

The `DataFrame.fillna()` function replaces NA with a spefic value or a specified method. Check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) for details.

- Pay attention to the warning message - Downcasting means converting from a general type (object) to a more specific type (int, float, etc).


In [25]:
#pd.set_option('future.no_silent_downcasting', True)

df.fillna(0)         # return a copy of df, with NA replaced as 0 

/var/folders/_1/w3rmdmrx14qdr85c4l_crhq80000gn/T/ipykernel_17091/820268198.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0)         # return a copy of df, with NA replaced as 0


,0,1,2,3
0,1.0,0.0,2,0
1,2.0,3.0,5,0
2,0.0,4.0,6,0


**Notes**: to fill missing values with the valid values nearby, please explore the `DataFrame.ffill()` (check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) for details) or the `DataFrame.bfill()` (check [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) for details) method.